In [1]:
## Requirements

pip install -q langchain langchain-core langchain-community
pip install -q google-generativeai langchain-google-genai
pip install -q python-dotenv tiktoken

In [3]:
## LLM

# This function is to create a Gemini model object.
# We can reuse it instead of rewriting code wherever it is needed

from langchain_google_genai import ChatGoogleGenerativeAI

def get_llm(temperature=0.6):
    """
    Returns a Gemini model using LangChain wrapper.
    You can reduce the usage of tokens using `max_output_tokens` if your expected responses are shorter
    in-order to optimize API usage for generation and cost cutting.
    """
    return ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=temperature,
        max_output_tokens=1024 # Reduced from 4096 for resource optimization (token usage reduction) detaily mentioned  in the document
    )

In [4]:
## Public Domain Validation

# checks if the story belongs to the public domain.
# Here I don't use a fixed list. Instead, Gemini reasons about copyright status.

# A simple cache memory for public domain checks
public_domain_cache = {}

def validate_inputs(source_story, target_universe):
    """
    Checks whether the given story belongs to public domain.
    Blocks if it is a non-public-domain works. 
     I made a cache to reduce API calls for repeated stories in-order to save from getting charged.
    """

    if not source_story or not target_universe:
        raise ValueError("Both source_story and target_universe must be provided.")

    # Check cache first
    if source_story in public_domain_cache:
        cached_result = public_domain_cache[source_story]
        if cached_result == "NO":
            raise ValueError(
                f"The work '{source_story}' does not appear to be in the public domain (cached result)."
            )
        elif cached_result == "UNCERTAIN":
            print(" Gemini is UNCERTAIN about public-domain status (cached result). Continue carefully.")
        return True

    llm = get_llm(temperature=0.2)

    prompt = f"""
You are checking the copyright status of fictional and literary works.

Work title: "{source_story}"

Decide whether this work is in the public domain.

Respond in this format only:

STATUS: YES / NO / UNCERTAIN
REASONING: short explanation.
"""

    response = llm.invoke(prompt)
    text = getattr(response, "content", str(response)).lower()

    if "status: no" in text:
        public_domain_cache[source_story] = "NO" # Cache the negative result
        raise ValueError(
            f"The work '{source_story}' does not appear to be in the public domain."
        )
    elif "status: uncertain" in text:
        public_domain_cache[source_story] = "UNCERTAIN" # Cache uncertain
        print(" Gemini is UNCERTAIN about public-domain status. Continue carefully.")
    else:
        public_domain_cache[source_story] = "YES" # Cache positive

    return True

In [5]:
## Extract

from langchain_core.prompts import PromptTemplate

def extract_narrative_essence(source_story):
    """ extracts the ESSENCE of the story without copying the original text. """

    llm = get_llm(temperature=0.4)

    prompt = PromptTemplate.from_template(""" Summarize only the ESSENCE of the work titled "{story}".

Do NOT copy original wording.

Return:
 main character arc
 central conflict
 key themes
 emotional tone
 moral core

Make it compact and neutral. """)

    chain = prompt | llm

    response = chain.invoke({"story": source_story})

    return response.content

In [6]:
## rules to build Universe

def build_target_universe(universe_description):
    """ Creates universe rules for ANY requested universe to make it look livelier.

    Gemini will take care of:
     physics logics / culture / tech level that is present in the targeted universe
     it determines the limitations and risks in-order to match the universe
     it avoids deus-ex-machina logic holes to make the story that as human creator """

    llm = get_llm(temperature=0.5)

    prompt = PromptTemplate.from_template("""
You are a universe-building expert.

User wants to reimagine a classic story in a new universe.

Universe description provided by user:
{universe}

Define clear rules for this universe:

Include:
 time period or technological level
 social or political structure
 power systems (magic, AI, corporations, governments, gods, etc.)
 constraints and costs (important to avoid easy solutions)
 tone and atmosphere
 what "success" or "victory" usually looks like here
 what is impossible in this universe

Important:
 keep it logically consistent
 avoid deus ex machina
 avoid stereotypes
""")

    chain = prompt | llm

    response = chain.invoke({"universe": universe_description})

    return response.content

In [7]:
## Character Mapping

def map_characters_any_universe(essence, universe_rules):
    """ Mapping the original story characters to the new universe with the proper logical by comparing the nature of the character from the original. """

    llm = get_llm(temperature=0.5)

    prompt = PromptTemplate.from_template("""
You will map characters from the original work into the new universe described.

Original story essence:
{essence}

Universe rules:
{rules}
""")

    chain = prompt | llm

    response = chain.invoke({
        "essence": essence,
        "rules": universe_rules
    })

    return response.content


In [8]:
## transformation of story Plot

def transform_plot_any_universe(essence, character_map, universe_rules):

    llm = get_llm(temperature=0.6)

    prompt = PromptTemplate.from_template("""
Transform the original story into the target universe.

Original essence:
{essence}

Character mappings:
{characters}

Universe rules:
{rules}

Requirements:
 preserve key themes and emotional arcs
 respect universe rules and constraints
 do not copy original text
 avoid sudden miracle solutions
 ensure motivations remain logical
 keep central conflict recognizable but reinterpreted

Output:
 structured plot outline with major acts or chapters
""")

    chain = prompt | llm

    response = chain.invoke({
        "essence": essence,
        "characters": character_map,
        "rules": universe_rules
    })

    return response.content


In [9]:
## Final story

def generate_final_story_any_universe(essence, character_map, universe_rules, plot_outline):

    llm = get_llm(temperature=0.75)

    prompt = PromptTemplate.from_template("""
Write a long narrative story (2–3 pages).

Do NOT copy any lines from the original story.

Use:
Essence:
{essence}

Character mappings:
{characters}

Universe rules:
{rules}

Plot outline:
{plot}

Ensure:
 world is immersive and consistent
 characters feel human and emotionally real
 conflict resolution follows universe logic
 no deus ex machina
 cultural sensitivity and respect
 thematic fidelity to original work
""")

    chain = prompt | llm

    response = chain.invoke({
        "essence": essence,
        "characters": character_map,
        "rules": universe_rules,
        "plot": plot_outline
    })

    return response.content


In [10]:
## Consistently Checking all the conditions

def refine_story(story, universe_rules, character_map):

    llm = get_llm(temperature=0.3)

    # This prompt is a general tepmlate that works on any universe
    prompt = PromptTemplate.from_template("""
Your task is to make a clean and correct version of the story below.

Fix the following issues:

 character name inconsistencies
 personality inconsistencies
 plot contradictions
 breaks in the internal logic of the universe
 violations of the world rules given
 sudden miracle solutions without explanation
 tone breaking moments

Do NOT:

 change the core plot
 change the ending meaning
 add new characters unless needed for logic
 remove important scenes
 do NOT use *** or any decorative separators
 do NOT use markdown symbols (*, _, #)
 write in plain paragraphs only

Story:
{story}

Universe Rules:
{rules}

Character Map:
{characters}

Return ONLY the corrected story, fully rewritten if needed.
Do not explain your changes.
""")

    chain = prompt | llm

    response = chain.invoke({
        "story": story,
        "rules": universe_rules,
        "characters": character_map
    })

    return response.content


In [11]:
## Pipeline

def run_pipeline():

    source_story = input("Enter the original story/work title: ")
    target_universe = input("Enter the new universe: ")

    validate_inputs(source_story, target_universe)

    essence = extract_narrative_essence(source_story)

    universe_rules = build_target_universe(target_universe)

    character_map = map_characters_any_universe(essence, universe_rules)

    plot_outline = transform_plot_any_universe(
        essence, character_map, universe_rules
    )

    draft_story = generate_final_story_any_universe(
        essence, character_map, universe_rules, plot_outline
    )

    final_story = refine_story(draft_story, universe_rules, character_map)

    print("\n FINAL STORY: \n")
    print(final_story)

In [1]:
## API Key 
import os

os.environ["GOOGLE_API_KEY"] = input("Enter your GOOGLE_API_KEY: ")

In [12]:
run_pipeline()

Enter the original story/work title:  Cinderella
Enter the new universe:  Corporate CEO selection system



 FINAL STORY: 

The stale, recycled air of the Fringe Zones clung to Elara Vance like a shroud, perpetually tinged with the metallic tang of industrial exhaust. Her Credit Score, a fragile 300, flickered on the wrist-mounted chronometer she had salvaged from a refuse channel, a constant reminder of her precarious existence within the OmniCorp Nexus. She moved through the labyrinthine corridors of the lower tiers, past cubicles stacked like hive cells, each flickering with the dim glow of bioluminescent strips. Her own living space was a testament to scarcity – a unit of compressed recyclables, its only luxury a cracked data-slate. As a Low-Tier Data Optimization Specialist, her daily grind involved monitoring dilapidated infrastructure, a task the Conglomerate’s main AIs deemed too low-yield to properly prioritize.

Yet, Elara possessed a quiet defiance. While her colleagues merely reported system failures, she sought remedies. She had an uncanny knack for coaxing life back into disca